In [25]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import os
import shutil
import matplotlib.pyplot as plt
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

os.environ["CUDA_VISIBLE_DEVICES"] = "0"  # Use GPU 0 (change to "1" for second GPU, etc.)

# Verify TensorFlow is using GPU
print("✅ Available GPUs:", tf.config.list_physical_devices('GPU'))


✅ Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
# Define the path to the zip file and the extraction directory
zip_path = "/content/drive/MyDrive/Virtual Operator/Dataset.zip"
extracted_dir = "/content/sample_data" # Directory to extract to

# Extract the dataset if it hasn't been extracted already

import zipfile
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
  zip_ref.extractall(extracted_dir)

"""
source_dir = "/kaggle/input/virtual-operator-proj-2025/Dataset"  # Your read-only dataset
dest_dir = "/kaggle/working/Dataset"  # Writable directory

if not os.path.exists(dest_dir):
    shutil.copytree(source_dir, dest_dir)
    print("✅ Dataset copied to writable directory.")
else:
    print("⚠️ Dataset already copied.")
"""

'\nsource_dir = "/kaggle/input/virtual-operator-proj-2025/Dataset"  # Your read-only dataset\ndest_dir = "/kaggle/working/Dataset"  # Writable directory\n\nif not os.path.exists(dest_dir):\n    shutil.copytree(source_dir, dest_dir)\n    print("✅ Dataset copied to writable directory.")\nelse:\n    print("⚠️ Dataset already copied.")\n'

In [31]:
dataset_dir = "/content/sample_data/Dataset"

for root, _, files in os.walk(dataset_dir):
    for file in files:
        file_path = os.path.join(root, file)
        try:
            with Image.open(file_path) as img:
                img.verify()  # Verify if image is valid
        except (IOError, SyntaxError):
            print(f"⚠️ Corrupt file detected and removed: {file_path}")
            os.remove(file_path)  # Delete corrupt file

In [32]:
for root, _, files in os.walk(dataset_dir):
    for file in files:
        if not file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')):
            print(f"⚠️ Found non-image file: {file}")

In [33]:
for root, _, files in os.walk(dataset_dir):
    for file in files:
        file_path = os.path.join(root, file)
        if file.lower().endswith(('.png', '.bmp', '.tiff', '.gif', '.webp')):
            try:
                img = Image.open(file_path).convert('RGB')
                new_path = file_path.rsplit(".", 1)[0] + ".jpg"
                img.save(new_path, "JPEG")
                os.remove(file_path)  # Remove original
                print(f"✅ Converted {file} to {new_path}")
            except Exception as e:
                print(f"❌ Failed to convert {file}: {e}")

In [40]:
dataset_dir = "/content/sample_data/Dataset"

# Hyper parameters
img_size = 300
batch_size = 32
epochs = 13
folds = 4  # Number of cross-validation folds
Lr = 0.0001

# STEP 1: SPLITTING DATASET (80% Train, 20% Test)
all_data_dir = os.listdir(dataset_dir)
train_test_dir = "split_dataset"
train_dir = os.path.join(train_test_dir, "train")
test_dir = os.path.join(train_test_dir, "test")

# Create train and test folders
if not os.path.exists(train_test_dir):
    os.makedirs(train_dir)
    os.makedirs(test_dir)

    # Loop through classes
    for class_name in all_data_dir:
        class_path = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_path):
            images = os.listdir(class_path)
            train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

            # Create class folders
            os.makedirs(os.path.join(train_dir, class_name))
            os.makedirs(os.path.join(test_dir, class_name))

            # Move images
            for img in train_images:
                shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))
            for img in test_images:
                shutil.copy(os.path.join(class_path, img), os.path.join(test_dir, class_name, img))

# STEP 2: SPLITTING TRAINING SET (70% Train, 30% Validation)
train_valid_dir = "train_validation_split"
train_subdir = os.path.join(train_valid_dir, "train")
valid_subdir = os.path.join(train_valid_dir, "valid")

if not os.path.exists(train_valid_dir):
    os.makedirs(train_subdir)
    os.makedirs(valid_subdir)

    for class_name in os.listdir(train_dir):
        class_path = os.path.join(train_dir, class_name)
        if os.path.isdir(class_path):
            images = os.listdir(class_path)
            train_images, valid_images = train_test_split(images, test_size=0.3, random_state=42)

            os.makedirs(os.path.join(train_subdir, class_name))
            os.makedirs(os.path.join(valid_subdir, class_name))

            for img in train_images:
                shutil.copy(os.path.join(class_path, img), os.path.join(train_subdir, class_name, img))
            for img in valid_images:
                shutil.copy(os.path.join(class_path, img), os.path.join(valid_subdir, class_name, img))

# STEP 3: DATA AUGMENTATION & PREPROCESSING

datagen = ImageDataGenerator(rescale=1.0 / 255,  # Normalize pixel values
    rotation_range=30,  # Rotate images up to 30 degrees
    width_shift_range=0.2,  # Shift images horizontally
    height_shift_range=0.2,  # Shift images vertically
    shear_range=0.2,  # Apply shear transformations
    zoom_range=0.2,  # Random zooming
    horizontal_flip=True,  # Flip images horizontally
    brightness_range=[0.8, 1.2],  # Adjust brightness
    fill_mode="nearest",  # Fill missing pixels after transformationsrescale=1.0 / 255)
                            )
train_generator = datagen.flow_from_directory(
    train_subdir, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical'
)
valid_generator = datagen.flow_from_directory(
    valid_subdir, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical'
)
test_generator = datagen.flow_from_directory(
    test_dir, target_size=(img_size, img_size), batch_size=batch_size, class_mode='categorical', shuffle=False
)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


CNN

In [ ]:
"""bold text
# STEP 4: BUILD CNN MODEL
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.4),
    Dense(train_generator.num_classes, activation='softmax')
])

# Compile Model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# STEP 5: CROSS-VALIDATION

for fold in range(folds):
    print(f"\n🚀 Cross-validation fold {fold + 1}/{folds}...\n")
    model.fit(train_generator, validation_data=valid_generator, epochs=epochs)

# STEP 6: TEST MODEL ON UNSEEN DATA
test_loss, test_acc = model.evaluate(test_generator)
print(f"\n🧪 Test Accuracy: {test_acc * 100:.2f}%")
print(f"🧪 Test Loss: {test_loss:.4f}")

# Define a counter
counter = 1
filename = f"/kaggle/working/my_model_{counter}.h5"

# Find the next available number
while os.path.exists(filename):
    counter += 1
    filename =  f"/kaggle/working/my_model_{counter}_acc{test_acc}.h5"

# Save the model
model.save(filename)
print(f"✅ Model saved as: {filename}")


# STEP 7: DISPLAY PREDICTIONS
import matplotlib.pyplot as plt
import numpy as np

# Get test images and labels
test_images, test_labels = next(test_generator)
predictions = model.predict(test_images)

# Show first 5 images and predictions
plt.figure(figsize=(12, 6))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(test_images[i])
    plt.title(f"Predicted: {np.argmax(predictions[i])}\nActual: {np.argmax(test_labels[i])}")
    plt.axis('off')

plt.show()"""

Inception

In [ ]:
"""
# STEP 4: LOAD PRETRAINED INCEPTIONV3 MODEL
base_model = InceptionV3(weights="imagenet", include_top=False, input_shape=(img_size, img_size, 3))

# Freeze base model layers
base_model.trainable = True # This prevents training the pre-trained layers

# Add Custom Classification Head
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Replaces Flatten
x = Dense(512, activation="relu")(x)
x = Dropout(0.4)(x)
output_layer = Dense(train_generator.num_classes, activation="softmax")(x)  # Adjusts for your classes

# Create Model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile Model
model.compile(optimizer=Adam(learning_rate=0.0001), loss="categorical_crossentropy", metrics=["accuracy"])

# STEP 5: CROSS-VALIDATION
for fold in range(folds):
    print(f"\n🚀 Cross-validation fold {fold + 1}/{folds}...\n")
    model.fit(train_generator, validation_data=valid_generator, epochs=epochs)

# STEP 6: TEST MODEL ON UNSEEN DATA
test_loss, test_acc = model.evaluate(test_generator)
print(f"\n🧪 Test Accuracy: {test_acc * 100:.2f}%")
print(f"🧪 Test Loss: {test_loss:.4f}")

# Define a counter
counter = 1
filename = f"/kaggle/working/my_model_{counter}_acc{test_acc:.4f}.h5"

# Find the next available number
while os.path.exists(filename):
    counter += 1
    filename = f"/kaggle/working/my_model_{counter}_acc{test_acc:.4f}.h5"

# Save the model
model.save(filename)
print(f"✅ Model saved as: {filename}")

output_path = f"/kaggle/working/{filename}"
shutil.move(filename, output_path)
print(f"✅ Model moved to {output_path}")

# STEP 7: DISPLAY PREDICTIONS
test_images, test_labels = next(test_generator)
predictions = model.predict(test_images)

# Show first 5 images and predictions
plt.figure(figsize=(12, 6))
for i in range(5):
    plt.subplot(1, 5, i+1)
    plt.imshow(test_images[i])
    plt.title(f"Predicted: {np.argmax(predictions[i])}\nActual: {np.argmax(test_labels[i])}")
    plt.axis("off")

plt.show()
"""

EfficientNet

In [ ]:

# STEP 4: LOAD PRETRAINED EFFICIENTNETB3 MODEL
base_model = EfficientNetB3(weights="imagenet", include_top=False, input_shape=(img_size, img_size, 3))

# Freeze base model layers
base_model.trainable = True  # You can set to False if you want to fine-tune only the top layers initially

# Add Custom Classification Head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.4)(x)
output_layer = Dense(train_generator.num_classes, activation="softmax")(x)

# Create Model
model = Model(inputs=base_model.input, outputs=output_layer)

# Compile Model
model.compile(optimizer=Adam(learning_rate=Lr), loss="categorical_crossentropy", metrics=["accuracy"])

# STEP 5: CROSS-VALIDATION
for fold in range(folds):
    print(f"\n🚀 Cross-validation fold {fold + 1}/{folds}...\n")
    model.fit(train_generator, validation_data=valid_generator, epochs=epochs)

# STEP 6: TEST MODEL ON UNSEEN DATA
test_loss, test_acc = model.evaluate(test_generator)
print(f"\n🧪 Test Accuracy: {test_acc * 100:.2f}%")
print(f"🧪 Test Loss: {test_loss:.4f}")

# Define a counter
counter = 1
filename = f"/kaggle/working/my_model_{counter}_acc{test_acc:.4f}.h5"

# Find the next available number
while os.path.exists(filename):
    counter += 1
    filename = f"/content/drive/MyDrive/Virtual Operator/Models/EfficientNet_{counter}_acc{test_acc:.4f}.h5"

# Save the model
model.save(filename)
print(f"✅ Model saved as: {filename}")

output_path = f"/kaggle/working/{filename}"
shutil.move(filename, output_path)
print(f"✅ Model moved to {output_path}")

# STEP 7: DISPLAY PREDICTIONS
test_images, test_labels = next(test_generator)
predictions = model.predict(test_images)

# Show first 5 images and predictions
plt.figure(figsize=(12, 6))
for i in range(5):
    plt.subplot(1, 5, i + 1)
    plt.imshow(test_images[i])
    plt.title(f"Predicted: {np.argmax(predictions[i])}\nActual: {np.argmax(test_labels[i])}")
    plt.axis("off")

plt.show()


In [ ]:

# 🔹 Load your trained model (adjust path as needed)
model_path = "/kaggle/working/my_model_1_acc0.9962.h5"  # Update with your model name
model = load_model(model_path)

# 🔹 Load and preprocess the image
img_path = "/kaggle/working/Dataset/Hero/IMG_20250318_011706_1.jpg"  # Replace with your image path
img = image.load_img(img_path, target_size=(img_size, img_size))  # Resize to match model input
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
img_array = img_array / 255.0  # Normalize

# 🔹 Predict
prediction = model.predict(img_array)
predicted_class = np.argmax(prediction, axis=1)[0]

# 🔹 Get class labels from generator
class_labels = list(train_generator.class_indices.keys())
predicted_label = class_labels[predicted_class]

# 🔹 Display
plt.imshow(img)
plt.title(f"Predicted: {predicted_label}")
plt.axis('off')
plt.show()
